In [2]:
# import library
import tensorflow as tf
import numpy as np
import sys, os,cv2
from scipy.misc import imread,imresize
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from imgaug import augmenters as iaa
import imgaug as ia
from scipy.ndimage import zoom
from sklearn.utils import shuffle
import matplotlib.animation as animation
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs,make_moons,make_swiss_roll
from IPython.display  import IFrame

plt.style.use('seaborn')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sess = tf.InteractiveSession()

In [26]:
# read all of the data
def read_all_images(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)

        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))

        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        return images
def read_labels(path_to_labels):
    """
    :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
    :return: an array containing the labels
    """
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels
    
test_images  = read_all_images("../../../DataSet/STL10/stl10_binary/test_X.bin")  / 255.0
test_labels  = read_labels    ("../../../DataSet/STL10/stl10_binary/test_y.bin")

label_encoder= OneHotEncoder(sparse=False,categories='auto')
test_labels  = label_encoder.fit_transform(test_labels.reshape((-1,1)))
print(test_images.shape)
print(test_labels.shape)

(8000, 96, 96, 3)
(8000, 10)


In [27]:
# shuffle and choose some 
test_images = test_images[np.random.permutation(range(8000))][:20,:,:,:]
print(test_images.shape)

(20, 96, 96, 3)


In [42]:
# show s u v
test_images_flat = np.reshape(test_images,(20,96*96,3))
s,U,V   = tf.svd(test_images_flat)
smin    = tf.reduce_min(s,1,keepdims=True)
smax    = tf.reduce_max(s,1,keepdims=True)
scaleds = (s - smin)/(smax - smin + 1e-8)

recon1 = U @ tf.matrix_diag(s)       @ tf.transpose(V,(0,2,1))
recon1 = recon1.eval()
recon2 = U @ tf.matrix_diag(scaleds) @ tf.transpose(V,(0,2,1))
recon2 = recon2.eval()
recon1_reshape = np.reshape(recon1,(20,96,96,3))
recon2_reshape = np.reshape(recon2,(20,96,96,3))

print(np.allclose(recon1,test_images_flat))
print(np.allclose(recon2,test_images_flat))
show()

True
False


In [43]:
def show(data):
    fig=plt.figure(figsize=(10, 10))
    columns = 4 ; rows = 5
    for i in range(1, columns*rows +1):
        fig.add_subplot(rows, columns, i)
        plt.imshow(data[i-1])
    plt.show()